In [89]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [90]:
def read_ubyte_file(file_path):
    with open(file_path, 'rb') as f:
        # Read the magic number
        magic_number = int.from_bytes(f.read(4), 'big')
        
        if magic_number == 2051:  # Magic number for images
            num_images = int.from_bytes(f.read(4), 'big')
            rows = int.from_bytes(f.read(4), 'big')
            cols = int.from_bytes(f.read(4), 'big')
            images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, rows, cols)
            return images
        
        elif magic_number == 2049:  # Magic number for labels
            num_labels = int.from_bytes(f.read(4), 'big')
            labels = np.frombuffer(f.read(), dtype=np.uint8)
            return labels
        
        else:
            raise ValueError('Invalid magic number in file header')

train_images_path = './data/train-images-idx3-ubyte'
train_labels_path = './data/train-labels-idx1-ubyte'

train_images = read_ubyte_file(train_images_path)
train_labels = read_ubyte_file(train_labels_path)


In [91]:
print(train_labels.shape)
print(train_labels[0])

(60000,)
5


In [92]:
print(train_images.shape)
# go from (60000, 28, 28) to (60000, 784)
X = train_images.reshape(train_images.shape[0],train_images.shape[1]*train_images.shape[2])
print(train_images[0])
X = X/255.0
print(X[0])


(60000, 28, 28)
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 25

In [93]:
def init_params():
    # each pixel from image needs to point to 10 hidden neurons
    W1 = np.random.randn(784, 10) * 0.01 
    #1 bias for each of the 10 hidden neurons
    b1 = np.random.randn(1, 10) * 0.01   
    
    # each of the 10 hidden neurons needs to point to each of the 10 output neurons
    W2 = np.random.randn(10, 10) * 0.01  
    # 1 bias for each of the 10 output neurons
    b2 = np.random.randn(1, 10) * 0.01   
    
    return W1, b1, W2, b2

In [94]:
def ReLU(x):
    return np.maximum(0, x)

In [95]:
def soft_max(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True))  # Subtract max for numerical stability
    return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)

In [96]:
def one_hot_encode(labels, num_classes):
    # Initialize a matrix of zeros with shape (number of labels, number of classes)
    one_hot = np.zeros((labels.size, num_classes))
    
    # Set the corresponding index to 1 for each label
    one_hot[np.arange(labels.size), labels] = 1
    
    return one_hot

Y = one_hot_encode(train_labels, 10)

In [97]:
def compute_loss(A2, Y):
    m = Y.shape[0]
    Y_int = np.argmax(Y, axis=1)

    log_probs = -np.log(A2[np.arange(m), Y_int])
    loss = np.sum(log_probs) / m
    return loss

In [98]:
def forward(X, W1, b1, W2, b2, Y):
    Z1 = np.dot(X, W1) + b1
    A1 = ReLU(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = soft_max(Z2)
    return Z1, A1, Z2, A2

In [99]:
def back_prop(X, Y, Z1, A1, Z2, A2, W1, W2, b1, b2, learning_rate):
    # used to find the normalized gradient
    # m = num of training examples in batch
    m = X.shape[0]
    dZ2 = A2 - Y
    
    # multiply the output of h1 by the change in the output
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    
    dA = np.dot(dZ2, W2.T)
    
    # relu derivative
    dZ1 = dA * (Z1 > 0)
    
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    
    return dW1, db1, dW2, db2

In [100]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

In [101]:
def train(X, Y, W1, b1, W2, b2, learning_rate, epochs):
    for i in range(epochs):
        # forward
        Z1, A1, Z2, A2 = forward(X, W1, b1, W2, b2, Y)
        loss = compute_loss(A2, Y)
        #backward
        dW1, db1, dW2, db2 = back_prop(X, Y, Z1, A1, Z2, A2, W1, W2, b1, b2, learning_rate)
        
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

        if i % 50 == 0:
            print(f'Epoch {i}: loss = {loss}')
    return W1, b1, W2, b2
        

In [109]:
W1, b1, W2, b2 = init_params()

In [112]:
W1, b1, W2, b2 = train(X, Y, W1, b1, W2, b2, 0.1, 1000)

Epoch 0: loss = 0.3364361134251093
Epoch 50: loss = 0.3314118984759991
Epoch 100: loss = 0.3267835500852011
Epoch 150: loss = 0.32248727134860666
Epoch 200: loss = 0.3185100010750274
Epoch 250: loss = 0.31480088803004047
Epoch 300: loss = 0.3113672855291566
Epoch 350: loss = 0.3081932827409766
Epoch 400: loss = 0.30525795666774447
Epoch 450: loss = 0.30253662615529436
Epoch 500: loss = 0.2999747429167423
Epoch 550: loss = 0.29757754976999934
Epoch 600: loss = 0.2953212435538508
Epoch 650: loss = 0.29317466295991984
Epoch 700: loss = 0.29114447796510184
Epoch 750: loss = 0.28921723492961104
Epoch 800: loss = 0.2873862807891883
Epoch 850: loss = 0.2856283276561296
Epoch 900: loss = 0.2839453501250728
Epoch 950: loss = 0.28233641333972836


In [114]:
def get_predictions(A2):
    return np.argmax(A2, axis=1)

def get_accuracy(predictions, Y):
    # Convert one-hot encoded Y to integer labels
    Y_int = np.argmax(Y, axis=1)
    return np.sum(predictions == Y_int) / Y_int.size

#returns A2
_, _, _, A2 = forward(X, W1, b1, W2, b2, Y)

# Get predictions from the output layer activations
predictions = get_predictions(A2)

# Calculate and print the accuracy
accuracy = get_accuracy(predictions, Y)
print(f'Accuracy: {accuracy:.5f}')

Accuracy: 0.92137
